In [8]:
import scala.util.Random
import scala.concurrent.{Future, Promise}
import scala.concurrent.ExecutionContext.Implicits.global
import scala.util.{Success, Failure}

def RandomNumberThreadExecutor(target: Int): Future[Int] = {
  val promise = Promise[Int]()
  val lock = new Object() // To synchronize the access to shared variables
  var stopThreads = false  // Shared flag to stop all threads
  var randomNumber = -1    // Shared random number that is generated by threads
  
  def generateRandomNumber(): Int = {
    randomNumber = new Random().nextInt(2000)
    randomNumber
  }

  // Thread 1
  val firstThread = new Thread(new Runnable {
    def run(): Unit = {
      while(randomNumber != target && !stopThreads) {
        randomNumber = generateRandomNumber()  // Update randomNumber
      }
      if (!stopThreads && randomNumber == target) {
        println("First thread has generated target " + target)
        promise.success(42)
        lock.synchronized {
          stopThreads = true
          lock.notifyAll() // Notify all threads to stop
        }
      }
    }
  })

  // Thread 2
  val secondThread = new Thread(new Runnable {
    def run(): Unit = {
      while(randomNumber != target && !stopThreads) {
        randomNumber = generateRandomNumber()  
      }
      if (!stopThreads && randomNumber == target) {
        println("Second thread has generated target " + target)
        promise.success(42)
        lock.synchronized {
          stopThreads = true
          lock.notifyAll() 
        }
      }
    }
  })

  // Thread 3
  val thirdThread = new Thread(new Runnable {
    def run(): Unit = {
      while(randomNumber != target && !stopThreads) {
        randomNumber = generateRandomNumber()  
      }
      if (!stopThreads && randomNumber == target) {
        println("Third thread has generated target " + target)
        promise.success(42)
        lock.synchronized {
          stopThreads = true
          lock.notifyAll() 
      }
    }
  })

  // Start all threads
  firstThread.start()
  secondThread.start()
  thirdThread.start()

  firstThread.join()
  secondThread.join()
  thirdThread.join()

  promise.future
}

val result: Future[Int] = RandomNumberThreadExecutor(1567)

result.onComplete {
  case Success(result) => println(s"Result: $result")
  case Failure(e) => println(s"Failed: ${e.getMessage}")
}

Third thread has generated target 1567
Result: 42


import scala.util.Random
import scala.concurrent.{Future, Promise}
import scala.concurrent.ExecutionContext.Implicits.global
import scala.util.{Success, Failure}
defined function RandomNumberThreadExecutor
result: Future[Int] = Success(value = 42)